In [1]:
import pandas as pd
from wrangle import get_codeup_data
from wrangle import combine_codeup_data
from wrangle import null_filler

# ACQUIRE
I will acquire the data for this project using __get_codeup_data__ function from my __wrangle.py__ file.
***

In [2]:
# using function to acquire data
logs, cohorts = get_codeup_data()

In [3]:
# previewing logs data
logs.head(2)

,date,time,page_viewed,user_id,cohort_id,ip
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61


In [4]:
# previewing cohorts data
cohorts.head(2)

,cohort_id,name,start_date,end_date,program_id
0,1,Arches,2014-02-04,2014-04-22,1
1,2,Badlands,2014-06-04,2014-08-22,1


# PREPARATION
I will prepare the data for the exploration necessary to answer the various questions I was asked.
***

#### Combine the data from the two separate DFs using the __combine_codeup_data__ function from my __wrangle.py__ file.

In [5]:
# combining DFs using function from local wrangle.py file
df = combine_codeup_data(logs, cohorts)

In [6]:
# previewing data 
df.head()

,page_viewed,user_id,cohort_id,ip,name,start_date,end_date,program_id
datetime,,,,,,,,
2018-01-26 09:55:03,/,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
2018-01-26 09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
2018-01-26 09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
2018-01-26 09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,2.0


#### I'm going to use .info to check if the column data types are appropriate and if there are any null values

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 719459 entries, 2018-01-26 09:55:03 to 2020-11-02 16:48:47
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   page_viewed  719458 non-null  object 
 1   user_id      719459 non-null  int64  
 2   cohort_id    674619 non-null  float64
 3   ip           719459 non-null  object 
 4   name         674619 non-null  object 
 5   start_date   674619 non-null  object 
 6   end_date     674619 non-null  object 
 7   program_id   674619 non-null  float64
dtypes: float64(2), int64(1), object(5)
memory usage: 49.4+ MB


- __Update data types if necessary for later work__
- Update missing values with special values that distinguish them from non-nulls
    - page_viewed: PageUnknown
    - cohort_id: 0
    - name: unknown
    - start_date and end_date: 99-99-9999
    - program_id: 0

#### I'm going to use my null_filler function from my wrangle.py file to fill null values with the values specified above.

In [8]:
# Using function from wrangle file to fill null values
null_filler(df)

In [9]:
# Checking to confirm if all nulls are removed
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 719459 entries, 2018-01-26 09:55:03 to 2020-11-02 16:48:47
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   page_viewed  719459 non-null  object 
 1   user_id      719459 non-null  int64  
 2   cohort_id    719459 non-null  object 
 3   ip           719459 non-null  object 
 4   name         719459 non-null  object 
 5   start_date   719459 non-null  object 
 6   end_date     719459 non-null  object 
 7   program_id   719459 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 49.4+ MB


#### I'm going to split the data science, web development, and unknown cohorts into separate DFs to make exploring them separately easier.

First I'll need to identify which program IDs belong with web development and which belong to data science.

I'll do this by cross-referencing the cohorts featured at <https://alumni.codeup.com> with the name column in my DF and take note of their program IDs.

In [10]:
# filtering for an Andromeda cohort row to see what program ID it is associated with
df[df.name == 'Andromeda'].head(1)

,page_viewed,user_id,cohort_id,ip,name,start_date,end_date,program_id
datetime,,,,,,,,
2019-03-18 16:01:40,assets/js/pdfmake.min.js.map,373,31,97.105.19.58,Andromeda,2019-03-18,2019-07-30,2.0


Andromeda is a web dev cohort.

program_id 2 indicates a web dev cohort.

In [11]:
# filtering for an Arches cohort row to see what program ID it is associated with
df[df.name == 'Arches'].head(1)

,page_viewed,user_id,cohort_id,ip,name,start_date,end_date,program_id
datetime,,,,,,,,
2018-01-26 10:14:47,/,11,1,97.105.19.61,Arches,2014-02-04,2014-04-22,1.0


Arches is a web dev cohort.

program_id 1 indicates a web dev cohort.

In [12]:
# filtering for an Apollo cohort row to see what program ID it is associated with
df[df.name == 'Apollo'].head(1)

,page_viewed,user_id,cohort_id,ip,name,start_date,end_date,program_id
datetime,,,,,,,,
2018-03-22 19:01:49,/,152,9,207.68.209.17,Apollo,2015-03-30,2015-07-29,4.0


Apollo is a web dev cohort.

program_id 4 indicates a web dev cohort.

In [13]:
# filtering for a Darden cohort row to see what program ID it is associated with
df[df.name == 'Darden'].head(1)

,page_viewed,user_id,cohort_id,ip,name,start_date,end_date,program_id
datetime,,,,,,,,
2020-07-13 14:34:44,/,678,59,76.201.20.193,Darden,2020-07-13,2021-01-12,3.0


Darden is a web dev cohort.

Program_id 3 indicates a Data Science cohort.

After cross-referencing the cohorts featured at <https://alumni.codeup.com> with the name column in my DF, I was able to determine that
 - Web development cohorts correspond with program IDs 1, 2 and 4
 - Data science cohorts correspond with program ID 3

#### Now that I know what program IDs correspond to web dev and DS cohorts, I can split them accordingly.

#### I'll also be splitting the unknown cohort rows into their own data frame.

In [16]:
# creating web dev DF
wd = df[(df.program_id == 1) | (df.program_id == 2) | (df.program_id == 4)]

# creating data science DF
ds = df[df.program_id == 3]

# creating DF for unknown cohort rows
unk = df[df.program_id == 0]